In [1]:
import sys
import os
import subprocess
from math import *
import numpy as np
from datetime import *
from pylab import *
import matplotlib as mpl
from matplotlib.widgets import Slider, Button, RadioButtons
import matplotlib.pyplot as plt
from astropy.table import Table, Column 
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.patches as patches
import scipy.misc as scimisc
import random
from datetime import datetime
import json

# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# !pip install imgaug

import imageio
import imgaug as ia
import imgaug.augmenters as iaa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib


In [3]:
import numpy as np
import imgaug as ia
import imgaug.augmenters as iaa


ia.seed(100)

seq = iaa.Sequential([

    iaa.Sometimes(0.5, iaa.GaussianBlur(sigma=(0, 0.1))),
    iaa.GammaContrast(gamma=(0.97,1.03)),
    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 3), per_channel=0.5),
    iaa.Add((-5, 5), per_channel=0.5),
    iaa.Multiply((0.8, 1.2), per_channel=0.5),
    iaa.Sometimes(0.1, iaa.Grayscale(alpha=(0.0, 1.0))),
    iaa.Sometimes(0.50, iaa.Grayscale(alpha=1.)),
    iaa.Affine(
        rotate=(0, 359),
        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
        translate_percent={"x": (-0.15, 0.15), "y": (-0.15, 0.15)})
    ], random_order=True)


seqGray = iaa.Sequential([

    iaa.Sometimes(0.5, iaa.GaussianBlur(sigma=(0, 0.1))),
    iaa.GammaContrast(gamma=(0.97,1.03)),
    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 3), per_channel=0.5),
    iaa.Add((-5, 5), per_channel=0.5),
    iaa.Multiply((0.8, 1.2), per_channel=0.5),
    iaa.Affine(
        rotate=(0, 359),
        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
        translate_percent={"x": (-0.15, 0.15), "y": (-0.15, 0.15)})
    ], random_order=True)

In [4]:
data = np.load('./data_128x128_RGB.npz')
images_ = data['images'].astype(np.uint8)
labels_ = data['labels'] 
data.close() 
del data

In [5]:
from sklearn.model_selection import train_test_split

ix, = np.where(labels_[1]>45)

images_c = images_[ix]
labels_c = labels_[1][ix]
pgcIDs_c = labels_[0][ix]

images_train, images_test, labels_train, labels_test, pgcIDs_train,  pgcIDs_test = train_test_split(images_c, labels_c, pgcIDs_c, test_size=0.40)

notUnique = np.isin(pgcIDs_test, pgcIDs_train)
Unique = np.logical_not(notUnique)

images_train_RGB = np.concatenate((images_train, images_test[notUnique]))
labels_train_RGB  = np.concatenate((labels_train, labels_test[notUnique]))
pgcIDs_train_RGB = np.concatenate((pgcIDs_train, pgcIDs_test[notUnique]))

images_test_RGB = images_test[Unique]
labels_test_RGB = labels_test[Unique]
pgcIDs_test_RGB = pgcIDs_test[Unique]

del images_
del labels_

del images_c
del labels_c
del pgcIDs_c

In [6]:
data = np.load('./data_128x128_g.npz')
images_ = data['images'].astype(np.uint8)
labels_ = data['labels'] 
data.close() 
del data

pgcIDs_ = labels_[0]
labels_ = labels_[1]

inTrain = np.isin(pgcIDs_, pgcIDs_train_RGB)
inTest = np.isin(pgcIDs_, pgcIDs_test_RGB)

images_train_gray = images_[inTrain]
labels_train_gray = labels_[inTrain]
pgcIDs_train_gray = pgcIDs_[inTrain]

images_test_gray = images_[inTest]
labels_test_gray = labels_[inTest]
pgcIDs_test_gray = pgcIDs_[inTest]


del pgcIDs_
del labels_
del images_

In [7]:
data = np.load('./data_128x128_r.npz')
images_ = data['images'].astype(np.uint8)
labels_ = data['labels'] 
data.close() 
del data

pgcIDs_ = labels_[0]
labels_ = labels_[1]

inTrain = np.isin(pgcIDs_, pgcIDs_train_RGB)
inTest = np.isin(pgcIDs_, pgcIDs_test_RGB)

images_train_gray = np.concatenate((images_train_gray, images_[inTrain]))
labels_train_gray = np.concatenate((labels_train_gray, labels_[inTrain]))
pgcIDs_train_gray = np.concatenate((pgcIDs_train_gray, pgcIDs_[inTrain]))

images_test_gray = np.concatenate((images_test_gray, images_[inTest]))
labels_test_gray = np.concatenate((labels_test_gray, labels_[inTest]))
pgcIDs_test_gray = np.concatenate((pgcIDs_test_gray, pgcIDs_[inTest]))


del pgcIDs_
del labels_
del images_

In [9]:
data = np.load('./data_128x128_i.npz')
images_ = data['images'].astype(np.uint8)
labels_ = data['labels'] 
data.close() 
del data

pgcIDs_ = labels_[0]
labels_ = labels_[1]

inTrain = np.isin(pgcIDs_, pgcIDs_train_RGB)
inTest = np.isin(pgcIDs_, pgcIDs_test_RGB)

images_train_gray = np.concatenate((images_train_gray, images_[inTrain]))
labels_train_gray = np.concatenate((labels_train_gray, labels_[inTrain]))
pgcIDs_train_gray = np.concatenate((pgcIDs_train_gray, pgcIDs_[inTrain]))

images_test_gray = np.concatenate((images_test_gray, images_[inTest]))
labels_test_gray = np.concatenate((labels_test_gray, labels_[inTest]))
pgcIDs_test_gray = np.concatenate((pgcIDs_test_gray, pgcIDs_[inTest]))

del pgcIDs_
del labels_
del images_

In [10]:
def esn_shuffle(array, seed=0):
        np.random.seed(seed)
        np.random.shuffle(array)
        return array

In [11]:
N_testGray = len(pgcIDs_test_gray)
N_trainGray = len(pgcIDs_train_gray)

N_testGray, N_trainGray

(3384, 129060)

In [13]:
indx = np.arange(N_testGray)
indx = esn_shuffle(indx, seed=0)

images_test_gray = images_test_gray[indx]
labels_test_gray = labels_test_gray[indx]
pgcIDs_test_gray = pgcIDs_test_gray[indx]

In [14]:
indx = np.arange(N_trainGray)
indx = esn_shuffle(indx, seed=0)

images_train_gray = images_train_gray[indx]
labels_train_gray = images_train_gray[indx]
pgcIDs_train_gray = images_train_gray[indx]

In [ ]:
np.savez_compressed('./Gray_128x128_train_000.npz', 
                    images=images_train_gray, 
                    labels=labels_train_gray, 
                    pgcIDs=pgcIDs_train_gray)
np.savez_compressed('./Gray_128x128_test_000.npz', 
                    images=images_test_gray, 
                    labels=labels_test_gray, 
                    pgcIDs=pgcIDs_test_gray)

In [ ]:
np.savez_compressed('./RGB_128x128_train_000.npz', 
                    images=images_train_RGB, 
                    labels=labels_train_RGB, 
                    pgcIDs=pgcIDs_train_RGB)
np.savez_compressed('./RGB_128x128_test_000.npz', 
                    images=images_test_RGB, 
                    labels=labels_test_RGB, 
                    pgcIDs=pgcIDs_test_RGB)

In [ ]:
import copy 
t1 = datetime.now()


for j in range(25):

  # images_train_aug = copy.deepcopy(images_train)
  # labels_train_aug = copy.deepcopy(labels_train)
  # pgcIDs_train_aug = copy.deepcopy(pgcIDs_train)

  # images_test_aug = copy.deepcopy(images_test)
  # labels_test_aug = copy.deepcopy(labels_test)
  # pgcIDs_test_aug = copy.deepcopy(pgcIDs_test)


  for i in range(1):
    
    dataAug = seq(images=images_train)
    images_train_aug = dataAug # np.concatenate((images_train_aug, dataAug)) 
    labels_train_aug = labels_train # np.concatenate((labels_train_aug, labels_train)) 
    pgcIDs_train_aug = pgcIDs_train # np.concatenate((pgcIDs_train_aug, pgcIDs_train)) 

    dataAug = seq(images=images_test)
    images_test_aug = dataAug # np.concatenate((images_test_aug, dataAug)) 
    labels_test_aug = labels_test # np.concatenate((labels_test_aug, labels_test)) 
    pgcIDs_test_aug = pgcIDs_test # np.concatenate((pgcIDs_test_aug, pgcIDs_test)) 


    t2 =  datetime.now()
    print("Execution time: ", i, t2-t1)

  np.savez_compressed('/content/drive/My Drive/Colab Notebooks/Tsclased_128x128_trainaug_'+str(j)+'.npz', images=images_train_aug, labels=labels_train_aug, pgcIDs=pgcIDs_train_aug)
  np.savez_compressed('/content/drive/My Drive/Colab Notebooks/Tsclased_128x128_testaug_'+str(j)+'.npz', images=images_test_aug, labels=labels_test_aug, pgcIDs=pgcIDs_test_aug)
  print('save #', j)

Execution time:  0 0:04:52.428852
save # 0
Execution time:  0 0:09:44.503527
save # 1
Execution time:  0 0:14:40.173470
save # 2
Execution time:  0 0:19:33.536470
save # 3
Execution time:  0 0:25:15.983512
save # 4
Execution time:  0 0:30:07.267499
save # 5
Execution time:  0 0:35:29.748691
save # 6
Execution time:  0 0:40:18.888320
save # 7
Execution time:  0 0:45:02.920305
save # 8
Execution time:  0 0:49:56.648657
save # 9
Execution time:  0 0:55:35.584290
save # 10
Execution time:  0 1:01:11.237501
save # 11
Execution time:  0 1:06:04.594874
save # 12
Execution time:  0 1:11:52.217900
save # 13
Execution time:  0 1:17:23.443271
save # 14
Execution time:  0 1:22:25.642447
save # 15
Execution time:  0 1:28:03.952067
save # 16
Execution time:  0 1:33:07.103613
save # 17
Execution time:  0 1:38:39.679928
save # 18
Execution time:  0 1:44:23.401976
save # 19
Execution time:  0 1:49:29.561257
save # 20
Execution time:  0 1:54:53.302962
save # 21
Execution time:  0 2:00:05.356266
save # 2